In [1]:
import pandas as pd
from pandas import option_context

In [2]:
import sklearn.metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [3]:
source_path = '../../datasets/IoT_Network_Intrusion.csv'

In [4]:
df = pd.read_csv(source_path, header=0)

In [5]:
df.head()

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25/07/2019 03:25:53 AM,75,1,1,...,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,Anomaly,Mirai,Mirai-Ackflooding
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,26/05/2019 10:11:06 PM,5310,1,2,...,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,Anomaly,DoS,DoS-Synflooding
2,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11/07/2019 01:24:48 AM,141,0,3,...,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,Anomaly,Scan,Scan Port OS
3,192.168.0.13-192.168.0.16-9020-52964-6,192.168.0.16,52964,192.168.0.13,9020,6,04/09/2019 03:58:17 AM,151,0,2,...,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,Anomaly,Mirai,Mirai-Hostbruteforceg
4,192.168.0.1-239.255.255.250-36763-1900-17,192.168.0.1,36763,239.255.255.250,1900,17,10/09/2019 01:41:18 AM,153,2,1,...,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,Anomaly,Mirai,Mirai-Hostbruteforceg


In [6]:
df.dtypes

Flow_ID      object
Src_IP       object
Src_Port      int64
Dst_IP       object
Dst_Port      int64
             ...   
Idle_Max    float64
Idle_Min    float64
Label        object
Cat          object
Sub_Cat      object
Length: 86, dtype: object

In [7]:
#df.pop('Timestamp')
#df.pop('Flow_ID')
#df.pop('Src_IP')
#df.pop('Dst_IP')
#df.pop('Label')
#df.pop('Sub_Cat')
#df = df.drop(columns=['Timestamp', 'Flow_ID', 'Src_IP', 'Dst_IP', 'Label', 'Sub_Cat'])
df = df.drop(['Timestamp', 'Flow_ID', 'Src_IP', 'Dst_IP', 'Label', 'Sub_Cat'], axis=1)

In [8]:
df.head()

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Cat
0,10000,10101,17,75,1,1,982.0,1430.0,982.0,982.0,...,0,0.0,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,Mirai
1,2179,554,6,5310,1,2,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,DoS
2,52727,9020,6,141,0,3,0.0,2806.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,Scan
3,52964,9020,6,151,0,2,0.0,2776.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,Mirai
4,36763,1900,17,153,2,1,886.0,420.0,452.0,434.0,...,0,0.0,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,Mirai


In [9]:
le = preprocessing.LabelEncoder()
le.fit(df['Cat'])
df['Cat']=le.transform(df['Cat'])

In [10]:
df.head()

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Cat
0,10000,10101,17,75,1,1,982.0,1430.0,982.0,982.0,...,0,0.0,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,2
1,2179,554,6,5310,1,2,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,0
2,52727,9020,6,141,0,3,0.0,2806.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,4
3,52964,9020,6,151,0,2,0.0,2776.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,2
4,36763,1900,17,153,2,1,886.0,420.0,452.0,434.0,...,0,0.0,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,2


In [11]:
df.isnull().sum()

Src_Port         0
Dst_Port         0
Protocol         0
Flow_Duration    0
Tot_Fwd_Pkts     0
                ..
Idle_Mean        0
Idle_Std         0
Idle_Max         0
Idle_Min         0
Cat              0
Length: 80, dtype: int64

In [ ]:
valores_ausentes = df.isnull()

In [ ]:
valores_ausentes

In [ ]:
with pd.option_context('mode.use_inf_as_na',True):
    df = df.dropna()

In [12]:
df.dtypes

Src_Port           int64
Dst_Port           int64
Protocol           int64
Flow_Duration      int64
Tot_Fwd_Pkts       int64
                  ...   
Idle_Mean        float64
Idle_Std         float64
Idle_Max         float64
Idle_Min         float64
Cat                int64
Length: 80, dtype: object

In [14]:
labels = df.pop('Cat')
scaler = StandardScaler()
scaler.fit(df.values)
df = scaler.transform(df.values)

df = pd.DataFrame(df)
df['Cat'] = labels

KeyError: 'Cat'

In [15]:
df.head()

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Fwd_Act_Data_Pkts,Fwd_Seg_Size_Min,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
0,10000,10101,17,75,1,1,982.0,1430.0,982.0,982.0,...,1,0,0.0,0.0,0.0,0.0,75.0,0.000000,75.0,75.0
1,2179,554,6,5310,1,2,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0
2,52727,9020,6,141,0,3,0.0,2806.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,70.5,0.707107,71.0,70.0
3,52964,9020,6,151,0,2,0.0,2776.0,0.0,0.0,...,0,0,0.0,0.0,0.0,0.0,151.0,0.000000,151.0,151.0
4,36763,1900,17,153,2,1,886.0,420.0,452.0,434.0,...,2,0,0.0,0.0,0.0,0.0,76.5,0.707107,77.0,76.0


In [ ]:
data_y = df.pop('Cat').values
data_X = df.values

In [ ]:
data_y

In [ ]:
data_X

In [ ]:
training_X, test_X, training_y, test_y = train_test_split(data_X, data_y, test_size=0.3)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_features="sqrt", 
                            criterion="gini", max_depth=100, 
                            min_samples_split=2, min_samples_leaf=1)
rf.fit(training_X, training_y)
rf_predict = rf.predict(test_X)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1, weights='uniform' , algorithm='kd_tree')
knn.fit(training_X, training_y)
knn_predict = knn.predict(test_X)

In [ ]:
acc = sklearn.metrics.accuracy_score(test_y, predictions)
print('Accuracy: {}'.format(acc))

acc_balanced = sklearn.metrics.balanced_accuracy_score(test_y, predictions)
print('Accuracy balanced: {}'.format(acc_balanced))

precision = sklearn.metrics.precision_score(test_y, predictions, average=Nome)
print('Precision: {}'.format(acc_balanced))

recall = sklearn.metrics.recall_score(test_y, predictions, average=Nome)
print('Recall: {}'.format(recall))

f1_score = sklearn.metrics.f1_score(test_y, predictions, average=Nome)
print('F1_score: {}'.format(f1_score))